In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import re
import seaborn as sns
sns.set(color_codes=True)

# pip install tabula-py
# потребуется установка Java https://www.java.com/en/download/
# conda install -c conda-forge jpype1
import tabula
print(pd.__version__)
print(tabula.__version__)

2.2.2
2.10.0


In [2]:
df_pdf = tabula.read_pdf('kcp-mag-2021 v2.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = pd.concat(df_pdf, ignore_index=True)

In [3]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9
0,Код,Направление подготовки\rНаправленность (профил...,Дополнительная информация,Контрольные цифры приема,Грант\r(за счет средств\rАкадемии),По договорам об оказании платных образовательн...,NaN,NaN,NaN,NaN
1,NaN,NaN,Общие условия,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,очное,заочное,очно-\rзаочное,очное,заочное,очно-заочное,NaN,NaN
3,1,2,3,5,6,7,NaN,8,9,10
4,УСЛОВИЯ ПОСТУПЛЕНИЯ:\rФедеральное государствен...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,09.04.03,Прикладная информатика\r- по программе магистр...,NaN,0,0,0,0,Цифры приема будут\rразмещены на сайте\rАкадем...,0,0
6,09.04.03,Прикладная информатика\r- по программе магистр...,Образовательная программа\rреализуется совмест...,0,0,0,0,0,0,NaN
7,37.04.01,Психология\r- по программе магистратуры в пред...,NaN,10,0,0,0,0,Цифры приема будут\rразмещены на сайте\rАкадем...,NaN
8,37.04.01,Психология\r- по программе магистратуры в пред...,NaN,5,0,0,0,0,NaN,NaN
9,38.04.01,Экономика\r- по программе магистратуры в преде...,NaN,15,0,0,0,0,0,NaN


In [4]:
df.iloc[:,:2]

,0,1
0,Код,Направление подготовки\rНаправленность (профил...
1,NaN,NaN
2,NaN,NaN
3,1,2
4,УСЛОВИЯ ПОСТУПЛЕНИЯ:\rФедеральное государствен...,NaN
...,...,...
163,42.04.05,Медиакоммуникации\r- по программе магистратуры...
164,42.04.02,Журналистика\r- по программе магистратуры в пр...
165,46.04.01,История\r- по программе магистратуры в предела...
166,46.04.01,История\r- по программе магистратуры в предела...


In [5]:
df_mos = df[ df[1].str.contains('по программе|по совокупности', na=False) ].reset_index()
df_mos.tail()

print('дубликаты: {0}' .format( df_mos[[0,1]].duplicated().sum() ))

df_mos = df_mos[[0,1]].copy().drop_duplicates()
df_mos

дубликаты: 0


,0,1
0,09.04.03,Прикладная информатика\r- по программе магистр...
1,09.04.03,Прикладная информатика\r- по программе магистр...
2,37.04.01,Психология\r- по программе магистратуры в пред...
3,37.04.01,Психология\r- по программе магистратуры в пред...
4,38.04.01,Экономика\r- по программе магистратуры в преде...
...,...,...
138,42.04.05,Медиакоммуникации\r- по программе магистратуры...
139,42.04.02,Журналистика\r- по программе магистратуры в пр...
140,46.04.01,История\r- по программе магистратуры в предела...
141,46.04.01,История\r- по программе магистратуры в предела...


In [46]:
df_one = df_mos[df_mos[1].str.contains('по программе магистратуры')]
df_one.head(3)

,0,1
0,09.04.03,Прикладная информатика\r- по программе магистр...
1,09.04.03,Прикладная информатика\r- по программе магистр...
2,37.04.01,Психология\r- по программе магистратуры в пред...


In [58]:
df_one.iloc[0,0] # в столбец КОД

'09.04.03'

In [59]:
df_one.iloc[0,1] # в остальные 3 столбца

'Прикладная информатика\r- по программе магистратуры в пределах направления\rподготовки:\rЦифровые технологии в экономике (ФИТАД ЭМИТ)'

In [60]:
df_one.iloc[0,1].split('\r',1)

['Прикладная информатика',
 '- по программе магистратуры в пределах направления\rподготовки:\rЦифровые технологии в экономике (ФИТАД ЭМИТ)']

In [61]:
df_one.iloc[0,1].split('\r',1)[0].strip()# в столбец Направление подготовки

'Прикладная информатика'

In [62]:
df_one.iloc[0,1].replace('\r', ' ')

'Прикладная информатика - по программе магистратуры в пределах направления подготовки: Цифровые технологии в экономике (ФИТАД ЭМИТ)'

In [68]:
df_one.iloc[0,1].replace('\r', ' ').split(':',1)

['Прикладная информатика - по программе магистратуры в пределах направления подготовки',
 ' Цифровые технологии в экономике (ФИТАД ЭМИТ)']

In [74]:
tmp_00 = df_one.iloc[0,1].replace('\r', ' ').split(':',1)[1]
tmp_00 = tmp_00.replace(re.findall("\\([ а-яa-zА-ЯA-Z0-9-: ]*\\)", tmp_00 )[-1],'')
tmp_00

' Цифровые технологии в экономике '

In [75]:
" ".join(tmp_00.split())

'Цифровые технологии в экономике'

In [76]:
tmp_00 = tmp_00.strip() # разместим в колонку 3
tmp_00 

'Цифровые технологии в экономике'

In [77]:
tmp_01 = df_one.iloc[0,1].replace('\r', ' ').split(':',1)[1]
tmp_01 

' Цифровые технологии в экономике (ФИТАД ЭМИТ)'

In [78]:
re.findall('\\(([ а-яa-zА-ЯA-Z0-9-: ]*)\\)', tmp_01 )[-1].strip() # разместим в колонку 4

'ФИТАД ЭМИТ'

In [80]:
df_one = df_mos[df_mos[1].str.contains('по программе магистратуры')]

def program_name(patern, tmp):
    tmp = tmp.replace('\r'," ").split(':',1)[1]
    #tmp = tmp.replace('  '," ")
    tmp = tmp.replace(re.findall(patern, tmp )[-1],'')
    " ".join(tmp.split())
    tmp = tmp.strip()
    return tmp

def facult_name(patern, tmp):
    tmp = tmp.replace('\r'," ")
    tmp = re.findall(patern, tmp )[-1].strip()
    return tmp

df_one_mos = pd.DataFrame([])
df_one_mos['код'] = df_one.iloc[:, 0]
df_one_mos['стандарт'] = df_one.iloc[:,1].map(lambda x: x.split('\r',1)[0].strip())
df_one_mos['программа'] =  df_one.iloc[:,1].map(lambda x: program_name("\\([ а-яa-zА-ЯA-Z0-9-: ]*\\)", x))
df_one_mos['подразделение'] =  df_one.iloc[:,1].map(lambda x: facult_name("\\(([ а-яa-zА-ЯA-Z0-9-: ]*)\\)", x))
df_one_mos['программа'] = df_one_mos['программа'].map(lambda x: re.sub('[ё]', 'е', x))




In [81]:
df_one_mos

,код,стандарт,программа,подразделение
0,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data ...,ФИТАД ЭМИТ
2,37.04.01,Психология,Психология управления,ИОН
3,37.04.01,Психология,Психология кризисных состояний и клиническая п...,ИОН
4,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ
...,...,...,...,...
138,42.04.05,Медиакоммуникации,Цифровые коммуникации и новые медиа,ИОН
139,42.04.02,Журналистика,Журналистика и медиатехнологии,ИГСУ
140,46.04.01,История,Социально-экономическая история России и мира,ИОН
141,46.04.01,История,Россия и Франция: историко-культурные связи,ИОН


In [90]:
df_multi = df_mos[df_mos[1].str.contains('по совокупности')]
df_multi

,0,1
33,38.04.02,Менеджмент\r- по совокупности программ магистр...
35,38.04.02,Менеджмент\r- по совокупности программ магистр...
65,38.04.04,Государственное и муниципальное управление\r- ...
106,40.04.01,Юриспруденция\r- по совокупности программ маги...
107,40.04.01,Юриспруденция\r- по совокупности программ маги...
108,40.04.01,Юриспруденция\r- по совокупности программ маги...


In [103]:
df_multi.index.values

array([ 33,  35,  65, 106, 107, 108])

In [104]:
tmp_3 = df_multi.loc[33][1].split('\r',1)[1]
tmp_3

'- по совокупности программ магистратуры в пределах\rнаправления подготовки:\r1. Управление рекламным и медиабизнесом (ИОМ)\r2. Цифровая трансформация в бизнесе (ИОМ)'

In [105]:
tmp_3 = " ".join(tmp_3.split())
tmp_3

'- по совокупности программ магистратуры в пределах направления подготовки: 1. Управление рекламным и медиабизнесом (ИОМ) 2. Цифровая трансформация в бизнесе (ИОМ)'

In [108]:
tmp_3 = re.split('\\d.',tmp_3)[1:]
tmp_3

[' Управление рекламным и медиабизнесом (ИОМ) ',
 ' Цифровая трансформация в бизнесе (ИОМ)']

In [114]:
df_multi = df_mos[df_mos[1].str.contains('по совокупности')]

def multi_transform(tmp):
    tmp_3 = tmp.iloc[1].split(':\r',1)[1]
    tmp_3 = " ".join(tmp_3.split())
    tmp_3 = re.split('\\d.',tmp_3)[1:]
    tmp_3 = np.char.replace(tmp_3,'\\r',' ')
    tmp_3 = np.char.replace(tmp_3,'.','')
    dic = {1: tmp.iloc[0],
           2: tmp.iloc[1].split('\\r',1)[0],
           3: tmp_3
          }
    df_tmp = pd.DataFrame(dic)
    df_tmp[4] = df_tmp[3].map(lambda x: facult_name("\\(([ а-яa-zА-ЯA-Z0-9-: ]*)\\)", x))
    df_tmp[3] = df_tmp[3].map(lambda x: x.replace(re.findall("\\([ а-яa-zА-ЯA-Z0-9-: ]*\\)", x )[-1],'').strip() )
    return df_tmp

tmp = [ multi_transform(df_multi.loc[idx]) for idx in df_multi.index.values]

#display(tmp)

df_multi_mos = pd.concat(tmp, ignore_index=True)
df_multi_mos.columns = df_one_mos.columns.values
#display(df_multi_mos.head(3))
display(df_multi_mos)


,код,стандарт,программа,подразделение
0,38.04.02,Менеджмент\r- по совокупности программ магистр...,Управление рекламным и медиабизнесом,ИОМ
1,38.04.02,Менеджмент\r- по совокупности программ магистр...,Цифровая трансформация в бизнесе,ИОМ
2,38.04.02,Менеджмент\r- по совокупности программ магистр...,Управление производственными системами,ИОМ
3,38.04.02,Менеджмент\r- по совокупности программ магистр...,Технологическое предпринимательство,ИОМ
4,38.04.04,Государственное и муниципальное управление\r- ...,Государственное управление и внешнеполитическа...,ФНБ ИПНБ
5,38.04.04,Государственное и муниципальное управление\r- ...,Государственное управление и национальная безо...,ФНБ ИПНБ
6,40.04.01,Юриспруденция\r- по совокупности программ маги...,Юрист в государственном аппарате,ЮФ ИПНБ
7,40.04.01,Юриспруденция\r- по совокупности программ маги...,Юрист в уголовном судопроизводстве,ЮФ ИПНБ
8,40.04.01,Юриспруденция\r- по совокупности программ маги...,"Юрист для бизнеса, власти и обеспечения прав ч...",ЮФ ИПНБ
9,40.04.01,Юриспруденция\r- по совокупности программ маги...,Правовое обеспечение контрольно-надзорной деят...,ЮФ ИПНБ


In [115]:
df_moscow = pd.concat([df_one_mos, df_multi_mos], ignore_index=True)
df_moscow.drop_duplicates(inplace=True)
df_moscow.head(3)

,код,стандарт,программа,подразделение
0,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data ...,ФИТАД ЭМИТ
2,37.04.01,Психология,Психология управления,ИОН


In [116]:
df_pdf = tabula.read_pdf('kcp-mag-filialu-2021 v2.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = pd.concat([df,df_temp],ignore_index=True)
#display(df[[0,1,2]].head(30))


display(df)


searchfor = ['Направление','Общие',"очное"]
mask = df[1].str.contains('|'.join(searchfor), na=False)|df[2].str.contains('|'.join(searchfor), na=False)

df = df[~mask][[0,1]].copy()
#df
#display(df.head(7))
mask_uslovie = df[0].str.contains("УСЛОВИЯ", na=False)
mask_uslovie
# #df[0] = df[0].str.replace('УСЛОВИЯ ПОСТУПЛЕНИЯ:\r','')
df[0] = df[0].str.replace('УСЛОВИЯ ПОСТУПЛЕНИЯ:\r','')
df[0] = df[0].map(lambda x: x.split('– программы')[0]) # внимание тут разное тире в разных строках!!! 
df[0] = df[0].map(lambda x: x.split('- программы')[0]) # внимание тут разное тире в разных строках!!! 
df[0] = df[0].map(lambda x: x.split('- филиал')[0])
mask_list = np.array(df[mask_uslovie].index)


with pd.option_context('display.max_rows', None):
         display(df)

#df.loc[mask_list]

,0,1,2,3,4,5,6,7
0,Код,Направление подготовки\rНаправленность (профил...,Контрольные цифры приема,По договорам об оказании платных образовательн...,NaN,NaN,NaN,NaN
1,NaN,Общие условия,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,очное,заочное,очно-заочное,очное,заочное,очно-заочное,NaN
3,УСЛОВИЯ ПОСТУПЛЕНИЯ:\rАлтайский филиал - прогр...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,07.04.04,Градостроительство\rпо программе магистратуры ...,0,0,0,0,0,Цифры приема\rбудут размещены\rна сайте Академ...
...,...,...,...,...,...,...,...,...
137,38.04.02,Менеджмент\rпо программе магистратуры в рамках...,0,2,0,0,0,NaN
138,38.04.04,Государственное и муниципальное управление\rпо...,10,4,0,Цифры приема\rбудут размещены\rна сайте Академ...,0,NaN
139,40.04.01,Юриспруденция\rпо совокупности программ магист...,15,0,0,0,0,NaN
140,40.04.01,Юриспруденция\rпо совокупности программ магист...,0,10,0,0,Цифры приема\rбудут размещены\rна сайте Академ...,0


,0,1
3,Алтайский филиал,NaN
4,07.04.04,Градостроительство\rпо программе магистратуры ...
5,37.04.01,Психология\rпо программе магистратуры в рамках...
6,38.04.01,Экономика\rпо программе магистратуры в рамках ...
7,38.04.04,Государственное и муниципальное управление\rпо...
8,38.04.08,Финансы и кредит\rпо совокупности программ маг...
9,40.04.01,Юриспруденция\rпо совокупности программ магист...
10,Брянский филиал,NaN
11,38.04.03,Управление персоналом\rпо программе магистрату...
12,38.04.04,Государственное и муниципальное управление\rпо...


In [118]:
def func_dic(df_test):
    dic = {} 
    code = df_test[0]
    dic.update({0: code})
    tmp = df_test[1]
    napravlenie = tmp.split('\r',1)[0]
    dic.update({1: napravlenie})
    tmp = tmp.split('\r',2)[2]
    tmp = " ".join(tmp.split())
    tmp = re.split('\\d.',tmp)[1:]
    tmp = np.char.replace(tmp,'\r',' ')
    tmp = np.char.replace(tmp,'.','')
    tmp = np.char.strip(tmp)
    dic.update({2: tmp})
    df_tmp = pd.DataFrame(dic)
    return df_tmp


df_multi_institute = []
mask_shape = mask_list.shape[0]
for i in range(mask_shape):
    if i < mask_shape-1:
        end = mask_list[i+1]-1
    else:
        end = df.index.max()
    
    df_fil = df.loc[mask_list[i]:end]
    
    tmp = [ func_dic(df_fil.loc[idx]) for idx in df_fil.iloc[1:].index.values]    
    df_multi = pd.concat(tmp, ignore_index=True)    
    df_multi[3] = df_fil.iloc[0][0].strip()

    
    df_multi = df_multi.drop_duplicates()
    df_multi_institute.append(df_multi)

df_multi_institute = pd.concat(df_multi_institute, ignore_index=True) 
df_multi_institute.columns = df_one_mos.columns.values

df_multi_institute['программа'] = df_multi_institute['программа'].str.replace('ё','е')

df_multi_institute.head(3)


,код,стандарт,программа,подразделение
0,07.04.04,Градостроительство,Современная урбанистика,Алтайский филиал
1,37.04.01,Психология,Прикладная социальная психология,Алтайский филиал
2,38.04.01,Экономика,Экономика здравоохранения,Алтайский филиал


In [119]:
df.loc[7][1].split('\r',2)

['Государственное и муниципальное управление',
 'по совокупности программ магистратуры в рамках направления подготовки',
 '1. Система государственного и муниципального управления\r2. Управление и экономика в сфере здравоохранения\r3. Управление образованием']

# Перечень всех программ магистратур Академии

In [120]:
magistratura = pd.concat([df_moscow,df_multi_institute],ignore_index=True)

with pd.option_context('display.max_rows', None):
         display(magistratura)

,код,стандарт,программа,подразделение
0,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data ...,ФИТАД ЭМИТ
2,37.04.01,Психология,Психология управления,ИОН
3,37.04.01,Психология,Психология кризисных состояний и клиническая п...,ИОН
4,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ
5,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
6,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ
7,38.04.01,Экономика,Цифровая экономика,ЭФ ЭМИТ
8,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
9,38.04.01,Экономика,Экономика и право,ЭФ ЭМИТ


In [121]:
tmp = magistratura[magistratura['код']=='38.04.01']

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
4,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ
5,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
6,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ
7,38.04.01,Экономика,Цифровая экономика,ЭФ ЭМИТ
8,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
9,38.04.01,Экономика,Экономика и право,ЭФ ЭМИТ
10,38.04.01,Экономика,Экономика медиаиндустрии,ЭФ ЭМИТ
11,38.04.01,Экономика,Экономика и контрольно-надзорная деятельность,ЭФ ЭМИТ
12,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ
13,38.04.01,Экономика,Бухгалтер-аналитик: корпоративная отчетность и финансовое моделирование,ФФБ


In [122]:
tmp = magistratura[magistratura['код']=='38.04.08']

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
96,38.04.08,Финансы и кредит,Прикладные корпоративные финансы,ВШФМ
97,38.04.08,Финансы и кредит,Денежно-кредитное и финансовое регулирование экономики,ФФБ
98,38.04.08,Финансы и кредит,Финансовый инженер,ФФБ
99,38.04.08,Финансы и кредит,Регулирование финансовых рынков и институтов,ИГСУ
100,38.04.08,Финансы и кредит,Финансы и инвестиции,ИФУР
171,38.04.08,Финансы и кредит,Государственные и муниципальные финансы,Алтайский филиал
172,38.04.08,Финансы и кредит,Финансовая диагностика и организационные технологии в бизнесе,Алтайский филиал
188,38.04.08,Финансы и кредит,Финансовый менеджмент,Волгоградский институт управления
197,38.04.08,Финансы и кредит,Финансовая экономика,Дальневосточный институт управления
224,38.04.08,Финансы и кредит,Корпоративные финансы,Нижегородский институт управления


# Перечень всех дубликатов программ магистратур Академии

In [123]:
tmp = magistratura[magistratura['программа'].duplicated(keep=False)].sort_values(by=['программа','код'])

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
95,38.04.05,Бизнес-информатика,Бизнес-аналитика,ФИТАД ЭМИТ
251,38.04.05,Бизнес-информатика,Бизнес-аналитика,Северо-Западный институт управления
72,38.04.04,Государственное и муниципальное управление,Государственная служба и кадровая политика,ИГСУ
228,38.04.04,Государственное и муниципальное управление,Государственная служба и кадровая политика,Петропавловский филиал
259,38.04.04,Государственное и муниципальное управление,Государственная служба и кадровая политика,Северо-Кавказский институт управления
...,...,...,...,...
144,40.04.01,"Юриспруденция\r- по совокупности программ магистратуры в пределах\rнаправления подготовки:\r1. Юрист в государственном аппарате (ЮФ ИПНБ)\r2. Юрист в уголовном судопроизводстве (ЮФ ИПНБ)\r3. Юрист для бизнеса, власти и обеспечения прав\rчеловека (ЮФ ИПНБ)\r4. Правовое обеспечение контрольно-надзорной\rдеятельности (ЮФ ИПНБ)\r5. Международная безопасность и права человека (ЮФ\rИПНБ)",Юрист в уголовном судопроизводстве,ЮФ ИПНБ
158,40.04.01,"Юриспруденция\r- по совокупности программ магистратуры в пределах\rнаправления подготовки:\r1. Гражданское право, гражданский процесс,\rарбитражный процесс (ЮФ ИПНБ)\r2. Магистр инновационного и предпринимательского\rправа в бизнес-проектах России (ЮФ ИПНБ)\r3. Правовые аспекты российского и международного\rбизнеса (ЮФ ИПНБ)\r4. Юрист в государственном аппарате (ЮФ ИПНБ)\r5. Юрист в сфере цифровой экономики (ЮФ ИПНБ)\r6. Юрист в уголовном судопроизводстве (ЮФ ИПНБ)\r7. Юрист для бизнеса, власти и обеспечения прав\rчеловека (ЮФ ИПНБ)\r8. Правовое обеспечение контрольно-надзорной\rдеятельности (ЮФ ИПНБ)\r9. Магистр частного права (ЮФ ИПНБ)\r10. Международная безопасность и права человека (ЮФ\rИПНБ)\r11. Международное транспортное право (ЮФ ИПНБ)\r12. Правовое обеспечение государственной службы\rказачества (ЮФ ИПНБ)",Юрист в уголовном судопроизводстве,ЮФ ИПНБ
145,40.04.01,"Юриспруденция\r- по совокупности программ магистратуры в пределах\rнаправления подготовки:\r1. Юрист в государственном аппарате (ЮФ ИПНБ)\r2. Юрист в уголовном судопроизводстве (ЮФ ИПНБ)\r3. Юрист для бизнеса, власти и обеспечения прав\rчеловека (ЮФ ИПНБ)\r4. Правовое обеспечение контрольно-надзорной\rдеятельности (ЮФ ИПНБ)\r5. Международная безопасность и права человека (ЮФ\rИПНБ)","Юрист для бизнеса, власти и обеспечения прав человека",ЮФ ИПНБ
159,40.04.01,"Юриспруденция\r- по совокупности программ магистратуры в пределах\rнаправления подготовки:\r1. Гражданское право, гражданский процесс,\rарбитражный процесс (ЮФ ИПНБ)\r2. Магистр инновационного и предпринимательского\rправа в бизнес-проектах России (ЮФ ИПНБ)\r3. Правовые аспекты российского и международного\rбизнеса (ЮФ ИПНБ)\r4. Юрист в государственном аппарате (ЮФ ИПНБ)\r5. Юрист в сфере цифровой экономики (ЮФ ИПНБ)\r6. Юрист в уголовном судопроизводстве (ЮФ ИПНБ)\r7. Юрист для бизнеса, власти и обеспечения прав\rчеловека (ЮФ ИПНБ)\r8. Правовое обеспечение контрольно-надзорной\rдеятельности (ЮФ ИПНБ)\r9. Магистр частного права (ЮФ ИПНБ)\r10. Международная безопасность и права человека (ЮФ\rИПНБ)\r11. Международное транспортное право (ЮФ ИПНБ)\r12. Правовое обеспечение государственной службы\rказачества (ЮФ ИПНБ)","Юрист для бизнеса, власти и обеспечения прав человека",ЮФ ИПНБ


# Перечень всех программ магистратур Академии по ЭКОНОМИКЕ

In [124]:
tmp = magistratura[magistratura['код']=='38.04.01'].sort_values(by='программа')
tmp

# with pd.option_context('display.max_colwidth', None):
#          display(tmp)

,код,стандарт,программа,подразделение
229,38.04.01,Экономика,Анализ и моделирование экономичяеских процессо...,Поволжский институт управления
12,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ
13,38.04.01,Экономика,Бухгалтер-аналитик: корпоративная отчетность и...,ФФБ
20,38.04.01,Экономика,Государственное регулирование экономики,ИГСУ
8,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
285,38.04.01,Экономика,Мировая экономика,Южно-Российский институт управления
284,38.04.01,Экономика,Налогообложение и налоговое администрирование,Южно-Российский институт управления
5,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
267,38.04.01,Экономика,Региональная экономика,Среднерусский институт управления
6,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ


# Перечень всех программ магистратур Академии по ФИНАНСАМ и КРЕДИТУ

In [125]:
tmp = magistratura[magistratura['код']=='38.04.08'].sort_values(by='программа')

with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
171,38.04.08,Финансы и кредит,Государственные и муниципальные финансы,Алтайский филиал
97,38.04.08,Финансы и кредит,Денежно-кредитное и финансовое регулирование экономики,ФФБ
224,38.04.08,Финансы и кредит,Корпоративные финансы,Нижегородский институт управления
96,38.04.08,Финансы и кредит,Прикладные корпоративные финансы,ВШФМ
99,38.04.08,Финансы и кредит,Регулирование финансовых рынков и институтов,ИГСУ
172,38.04.08,Финансы и кредит,Финансовая диагностика и организационные технологии в бизнесе,Алтайский филиал
197,38.04.08,Финансы и кредит,Финансовая экономика,Дальневосточный институт управления
98,38.04.08,Финансы и кредит,Финансовый инженер,ФФБ
188,38.04.08,Финансы и кредит,Финансовый менеджмент,Волгоградский институт управления
271,38.04.08,Финансы и кредит,Финансовый менеджмент,Среднерусский институт управления


In [126]:
# magistratura.to_excel("mag_2021.xlsx")